In [42]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import math

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

import pickle

import heapq


In [9]:
train_data=pd.read_csv('t_2_train.csv')
train_smiles=list(train_data.iloc[:,0])
train_labels=list(train_data.iloc[:,2])
test_data=pd.read_csv('t_2_test.csv')
test_smiles=list(test_data.iloc[:,0])
test_labels=list(test_data.iloc[:,2])

In [152]:
from rdkit.Chem import MACCSkeys
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem

# def get_finger(smiles):
#     all_mols=[]
#     for smi in list(smiles):
#         m=Chem.MolFromSmiles(smi)
#         all_mols.append(m)
#     #all_fps=[Chem.RDKFingerprint(mol) for mol in all_mols]
#     all_fingerprints =[MACCSkeys.GenMACCSKeys(mol).ToBitString() for mol in all_mols]
#     x = []
#     for finger in all_fingerprints:
#         x.append(np.fromiter(finger, dtype=int))
#     x = np.array(x)
#     return x
def get_finger(smiles):
    all_mols=[]
    for smi in smiles:
        m=Chem.MolFromSmiles(smi)
        all_mols.append(m)
    #all_fps=[Chem.RDKFingerprint(mol) for mol in all_mols]
    all_fingerprints1 =[MACCSkeys.GenMACCSKeys(mol).ToBitString() for mol in all_mols]
    all_fingerprints2 =[AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=1024).ToBitString() for mol in all_mols]
    #all_fingerprints=all_fingerprints1+all_fingerprints2
    #print(all_fingerprints)
    x = []
    for finger in all_fingerprints1:
        x.append(np.fromiter(finger, dtype=int))
    x = np.array(x)
    
    y = []
    for finger in all_fingerprints2:
        y.append(np.fromiter(finger, dtype=int))
    y = np.array(y)
    return x,y

In [11]:
for smi in list(train_smiles):
        m=Chem.MolFromSmiles(smi)
        try:
            Chem.RDKFingerprint(m) 
        except:
            print(smi)
            print(m)

In [20]:
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as ACC
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import f1_score as F1
def auc_roc(y_test1,pre):
    pre=[p for p in pre[:,1]]
    #print(pre)
    fpr,tpr,threshold = roc_curve(y_test1,pre)###计算真正率和假正率
    roc_auc = auc(fpr,tpr)
    #print('roc_auc:%2f'%(roc_auc))
    return roc_auc
def analyze(pre,real):
    AUC_test=auc_roc(real,pre)
    pre=np.argmax(pre,axis=1)
    #print(pre)
    m=confusion_matrix(pre,real, labels=[0,1], sample_weight=None)
    print(m)
    tp, fp, fn, tn = m.ravel()
    TPR=tp/(tp+fn)
    TNR=tn/(tn+fp)
    ACC_test=ACC(real,pre)
    #AUC_test=auc_roc(real,pre)
    MCC_test=MCC(real,pre)
    F1_test=F1(real,pre)
    print('敏感度TPR：%2f，特异度TNR：%2f，ACC：%2f，AUC：%2f,MCC：%2f,F1：%2f'%(TPR,TNR,ACC_test,AUC_test,MCC_test,F1_test))
    return ACC_test

In [155]:
# x_train=get_finger(train_smiles)
# x_test=get_finger(test_smiles)
x_test1,x_test2=get_finger(test_smiles)
x_train1,x_train2=get_finger(train_smiles)


In [156]:
df1 = pd.DataFrame(x_train1)
df2=pd.DataFrame(x_train2)
df=pd.concat([df1,df2],axis=1)
x_train=df.values#as_matrix()
df1 = pd.DataFrame(x_test1)
df2=pd.DataFrame(x_test2)
df=pd.concat([df1,df2],axis=1)
x_test=df.values#as_matrix()

In [157]:
num_samples=213

In [158]:
i=0
with open('t_2_rf02.pkl','rb') as rf:
        rf0=pickle.load(rf)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
print('train')
probarf_train0=rf0.predict_proba(train_data)
analyze(probarf_train0,train_label)
print('validation')
probarf_val0=rf0.predict_proba(val_data)
analyze(probarf_val0,val_label)
print('test')
probarf_test0=rf0.predict_proba(x_test)
analyze(probarf_test0,test_labels)


train
[[351   3]
 [  2 497]]
敏感度TPR：0.994334，特异度TNR：0.994000，ACC：0.994138，AUC：0.999722,MCC：0.987926,F1：0.994995
validation
[[ 43  30]
 [ 36 104]]
敏感度TPR：0.544304，特异度TNR：0.776119，ACC：0.690141，AUC：0.724731,MCC：0.326110,F1：0.759124
test
[[ 73  43]
 [ 35 115]]
敏感度TPR：0.675926，特异度TNR：0.727848，ACC：0.706767，AUC：0.728786,MCC：0.399857,F1：0.746753


0.706766917293233

In [161]:
pres=[]
for i in range(20):
    pre_singe=rf0.estimators_[i].predict_proba(val_data)
    pres.append(analyze(pre_singe,val_label))
#max_index_new0 = list(map(pres.index, heapq.nlargest(10,pres)))
#print(max_index_new0)

[[41 39]
 [38 95]]
敏感度TPR：0.518987，特异度TNR：0.708955，ACC：0.638498，AUC：0.613971,MCC：0.227363,F1：0.711610
[[45 53]
 [34 81]]
敏感度TPR：0.569620，特异度TNR：0.604478，ACC：0.591549，AUC：0.587049,MCC：0.168732,F1：0.650602
[[42 46]
 [37 88]]
敏感度TPR：0.531646，特异度TNR：0.656716，ACC：0.610329，AUC：0.594181,MCC：0.184783,F1：0.679537
[[36 52]
 [43 82]]
敏感度TPR：0.455696，特异度TNR：0.611940，ACC：0.553991，AUC：0.535519,MCC：0.066351,F1：0.633205
[[43 45]
 [36 89]]
敏感度TPR：0.544304，特异度TNR：0.664179，ACC：0.619718，AUC：0.606272,MCC：0.204522,F1：0.687259
[[40 39]
 [39 95]]
敏感度TPR：0.506329，特异度TNR：0.708955，ACC：0.633803，AUC：0.607642,MCC：0.215284,F1：0.708955
[[39 39]
 [40 95]]
敏感度TPR：0.493671，特异度TNR：0.708955，ACC：0.629108，AUC：0.601313,MCC：0.203164,F1：0.706320
[[35 51]
 [44 83]]
敏感度TPR：0.443038，特异度TNR：0.619403，ACC：0.553991，AUC：0.531220,MCC：0.061473,F1：0.636015
[[34 43]
 [45 91]]
敏感度TPR：0.430380，特异度TNR：0.679104，ACC：0.586854，AUC：0.554742,MCC：0.110079,F1：0.674074
[[ 38  29]
 [ 41 105]]
敏感度TPR：0.481013，特异度TNR：0.783582，ACC：0.671362，AUC：0.632297,M

In [167]:
i=1
with open('t_2_rf12.pkl','rb') as rf:
        rf1=pickle.load(rf)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
print('train')
probarf_train1=rf1.predict_proba(train_data)
analyze(probarf_train1,train_label)
print('validation')
probarf_val1=rf1.predict_proba(val_data)
analyze(probarf_val1,val_label)
print('test')
probarf_test1=rf1.predict_proba(x_test)
analyze(probarf_test1,test_labels)

train
[[334   1]
 [  2 516]]
敏感度TPR：0.994048，特异度TNR：0.998066，ACC：0.996483，AUC：0.999902,MCC：0.992634,F1：0.997101
validation
[[ 96   0]
 [  0 117]]
敏感度TPR：1.000000，特异度TNR：1.000000，ACC：1.000000，AUC：1.000000,MCC：1.000000,F1：1.000000
test
[[ 65  34]
 [ 43 124]]
敏感度TPR：0.601852，特异度TNR：0.784810，ACC：0.710526，AUC：0.730690,MCC：0.392822,F1：0.763077


0.7105263157894737

In [163]:
pres=[]
for i in range(20):
    pre_singe=rf1.estimators_[i].predict_proba(val_data)
    pres.append(analyze(pre_singe,val_label))
max_index_new1 = list(map(pres.index, heapq.nlargest(10,pres)))
print(max_index_new1)

[[ 85  12]
 [ 11 105]]
敏感度TPR：0.885417，特异度TNR：0.897436，ACC：0.892019，AUC：0.891426,MCC：0.782157,F1：0.901288
[[ 76  13]
 [ 20 104]]
敏感度TPR：0.791667，特异度TNR：0.888889，ACC：0.845070，AUC：0.839387,MCC：0.686572,F1：0.863071
[[ 87   6]
 [  9 111]]
敏感度TPR：0.906250，特异度TNR：0.948718，ACC：0.929577，AUC：0.927484,MCC：0.857721,F1：0.936709
[[83 18]
 [13 99]]
敏感度TPR：0.864583，特异度TNR：0.846154，ACC：0.854460，AUC：0.855369,MCC：0.708220,F1：0.864629
[[ 82  13]
 [ 14 104]]
敏感度TPR：0.854167，特异度TNR：0.888889，ACC：0.873239，AUC：0.875178,MCC：0.743784,F1：0.885106
[[79 18]
 [17 99]]
敏感度TPR：0.822917，特异度TNR：0.846154，ACC：0.835681，AUC：0.834535,MCC：0.668476,F1：0.849785
[[ 72  10]
 [ 24 107]]
敏感度TPR：0.750000，特异度TNR：0.914530，ACC：0.840376，AUC：0.832265,MCC：0.679517,F1：0.862903
[[ 80  11]
 [ 16 106]]
敏感度TPR：0.833333，特异度TNR：0.905983，ACC：0.873239，AUC：0.869658,MCC：0.743632,F1：0.887029
[[ 74  17]
 [ 22 100]]
敏感度TPR：0.770833，特异度TNR：0.854701，ACC：0.816901，AUC：0.812767,MCC：0.629186,F1：0.836820
[[ 87  13]
 [  9 104]]
敏感度TPR：0.906250，特异度TNR：0.888889

TypeError: 'range' object is not callable

In [165]:
i=2
with open('t_2_rf22.pkl','rb') as rf:
        rf2=pickle.load(rf)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
print('train')
probarf_train2=rf2.predict_proba(train_data)
analyze(probarf_train2,train_label)
print('validation')
probarf_val2=rf2.predict_proba(val_data)
analyze(probarf_val2,val_label)
print('test')
probarf_test2=rf2.predict_proba(x_test)
analyze(probarf_test2,test_labels)

train
[[357   2]
 [  1 493]]
敏感度TPR：0.997207，特异度TNR：0.995960，ACC：0.996483，AUC：0.999842,MCC：0.992785,F1：0.996967
validation
[[ 40  30]
 [ 34 109]]
敏感度TPR：0.540541，特异度TNR：0.784173，ACC：0.699531，AUC：0.740035,MCC：0.329159,F1：0.773050
test
[[ 69  43]
 [ 39 115]]
敏感度TPR：0.638889，特异度TNR：0.727848，ACC：0.691729，AUC：0.719702,MCC：0.364776,F1：0.737179


0.6917293233082706

In [166]:
pres=[]
for i in range(20):
    pre_singe=rf2.estimators_[i].predict_proba(val_data)
    pres.append(analyze(pre_singe,val_label))
#max_index_new2 = list(map(pres.index, heapq.nlargest(10,pres)))
#print(max_index_new2)

[[41 51]
 [33 88]]
敏感度TPR：0.554054，特异度TNR：0.633094，ACC：0.605634，AUC：0.593574,MCC：0.179896,F1：0.676923
[[35 48]
 [39 91]]
敏感度TPR：0.472973，特异度TNR：0.654676，ACC：0.591549，AUC：0.565526,MCC：0.124632,F1：0.676580
[[45 56]
 [29 83]]
敏感度TPR：0.608108，特异度TNR：0.597122，ACC：0.600939，AUC：0.602615,MCC：0.195702,F1：0.661355
[[36 49]
 [38 90]]
敏感度TPR：0.486486，特异度TNR：0.647482，ACC：0.591549，AUC：0.566984,MCC：0.130260,F1：0.674157
[[48 58]
 [26 81]]
敏感度TPR：0.648649，特异度TNR：0.582734，ACC：0.605634，AUC：0.615691,MCC：0.220348,F1：0.658537
[[37 48]
 [37 91]]
敏感度TPR：0.500000，特异度TNR：0.654676，ACC：0.600939，AUC：0.577338,MCC：0.150395,F1：0.681648
[[42 43]
 [32 96]]
敏感度TPR：0.567568，特异度TNR：0.690647，ACC：0.647887，AUC：0.631149,MCC：0.251067,F1：0.719101
[[40 54]
 [34 85]]
敏感度TPR：0.540541，特异度TNR：0.611511，ACC：0.586854，AUC：0.576026,MCC：0.145806,F1：0.658915
[[ 34  35]
 [ 40 104]]
敏感度TPR：0.459459，特异度TNR：0.748201，ACC：0.647887，AUC：0.608886,MCC：0.211287,F1：0.734982
[[ 39  35]
 [ 35 104]]
敏感度TPR：0.527027，特异度TNR：0.748201，ACC：0.671362，AUC：0.6376

In [138]:
i=3
with open('t_2_rf3.pkl','rb') as rf:
        rf3=pickle.load(rf)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
print('train')
probarf_train3=rf3.predict_proba(train_data)
analyze(probarf_train3,train_label)
print('validation')
probarf_val3=rf3.predict_proba(val_data)
analyze(probarf_val3,val_label)
print('test')
probarf_test3=rf3.predict_proba(x_test)
analyze(probarf_test3,test_labels)

train
[[336  14]
 [  3 500]]
敏感度TPR：0.991150，特异度TNR：0.972763，ACC：0.980070，AUC：0.997406,MCC：0.958962,F1：0.983284
validation
[[52 37]
 [41 83]]
敏感度TPR：0.559140，特异度TNR：0.691667，ACC：0.633803，AUC：0.699731,MCC：0.252212,F1：0.680328
test
[[ 64  38]
 [ 44 120]]
敏感度TPR：0.592593，特异度TNR：0.759494，ACC：0.691729，AUC：0.716450,MCC：0.355605,F1：0.745342


0.6917293233082706

In [97]:
pres=[]
for i in range(50):
    pre_singe=rf3.estimators_[i].predict_proba(val_data)
    pres.append(analyze(pre_singe,val_label))
max_index_new3 = list(map(pres.index, heapq.nlargest(10,pres)))
print(max_index_new3)

[[53 36]
 [40 84]]
敏感度TPR：0.569892，特异度TNR：0.700000，ACC：0.643192，AUC：0.634946,MCC：0.271404,F1：0.688525
[[34 32]
 [59 88]]
敏感度TPR：0.365591，特异度TNR：0.733333，ACC：0.572770，AUC：0.549462,MCC：0.106098,F1：0.659176
[[49 37]
 [44 83]]
敏感度TPR：0.526882，特异度TNR：0.691667，ACC：0.619718，AUC：0.609274,MCC：0.220917,F1：0.672065
[[50 40]
 [43 80]]
敏感度TPR：0.537634，特异度TNR：0.666667，ACC：0.610329，AUC：0.604391,MCC：0.205130,F1：0.658436
[[46 26]
 [47 94]]
敏感度TPR：0.494624，特异度TNR：0.783333，ACC：0.657277，AUC：0.638978,MCC：0.291430,F1：0.720307
[[55 30]
 [38 90]]
敏感度TPR：0.591398，特异度TNR：0.750000，ACC：0.680751，AUC：0.670699,MCC：0.345763,F1：0.725806
[[53 33]
 [40 87]]
敏感度TPR：0.569892，特异度TNR：0.725000，ACC：0.657277，AUC：0.647446,MCC：0.298088,F1：0.704453
[[46 32]
 [47 88]]
敏感度TPR：0.494624，特异度TNR：0.733333，ACC：0.629108，AUC：0.613978,MCC：0.234677,F1：0.690196
[[55 37]
 [38 83]]
敏感度TPR：0.591398，特异度TNR：0.691667，ACC：0.647887，AUC：0.641532,MCC：0.283420,F1：0.688797
[[53 40]
 [40 80]]
敏感度TPR：0.569892，特异度TNR：0.666667，ACC：0.624413，AUC：0.618280,MCC：0

In [139]:
i=4
with open('t_2_rf4.pkl','rb') as rf:
        rf4=pickle.load(rf)
val_data= x_train[i*num_samples:(i+1)*num_samples,:]
val_label= train_labels[i * num_samples:(i + 1) * num_samples]
train_data=np.concatenate([x_train[:i*num_samples,:],x_train[(i+1)*num_samples:,:]],axis=0)
train_label = np.concatenate([train_labels[:i*num_samples], train_labels[(i + 1) * num_samples:]], axis=0)
print('train')
probarf_train4=rf4.predict_proba(train_data)
analyze(probarf_train4,train_label)
print('validation')
probarf_val4=rf4.predict_proba(val_data)
analyze(probarf_val4,val_label)
print('test')
probarf_test4=rf4.predict_proba(x_test)
analyze(probarf_test4,test_labels)

train
[[337   6]
 [  5 505]]
敏感度TPR：0.985380，特异度TNR：0.988258，ACC：0.987104，AUC：0.999230,MCC：0.973171,F1：0.989226
validation
[[53 38]
 [37 85]]
敏感度TPR：0.588889，特异度TNR：0.691057，ACC：0.647887，AUC：0.687037,MCC：0.279542,F1：0.693878
test
[[ 71  40]
 [ 37 118]]
敏感度TPR：0.657407，特异度TNR：0.746835，ACC：0.710526，AUC：0.756476,MCC：0.402583,F1：0.753994


0.7105263157894737

In [140]:
print('test_all')
probarf_test=(probarf_test0+probarf_test1+probarf_test2+probarf_test3+probarf_test4)/5.0
#probarf=(probarf1+probarf2+probarf3+probarf4)/5.0
aa=analyze(probarf_test,test_labels)

test_all
[[ 61  25]
 [ 47 133]]
敏感度TPR：0.564815，特异度TNR：0.841772，ACC：0.729323，AUC：0.759054,MCC：0.426883,F1：0.786982


In [100]:
pres=[]
for i in range(50):
    pre_singe=rf4.estimators_[i].predict_proba(val_data)
    pres.append(analyze(pre_singe,val_label))
max_index_new4 = list(map(pres.index, heapq.nlargest(10,pres)))
print(max_index_new4)

[[53 42]
 [37 81]]
敏感度TPR：0.588889，特异度TNR：0.658537，ACC：0.629108，AUC：0.623713,MCC：0.245876,F1：0.672199
[[40 41]
 [50 82]]
敏感度TPR：0.444444，特异度TNR：0.666667，ACC：0.572770，AUC：0.555556,MCC：0.113058,F1：0.643137
[[51 40]
 [39 83]]
敏感度TPR：0.566667，特异度TNR：0.674797，ACC：0.629108，AUC：0.624661,MCC：0.241115,F1：0.677551
[[49 46]
 [41 77]]
敏感度TPR：0.544444，特异度TNR：0.626016，ACC：0.591549，AUC：0.583288,MCC：0.169393,F1：0.639004
[[44 35]
 [46 88]]
敏感度TPR：0.488889，特异度TNR：0.715447，ACC：0.619718，AUC：0.598509,MCC：0.208955,F1：0.684825
[[43 38]
 [47 85]]
敏感度TPR：0.477778，特异度TNR：0.691057，ACC：0.600939，AUC：0.584417,MCC：0.171793,F1：0.666667
[[46 37]
 [44 86]]
敏感度TPR：0.511111，特异度TNR：0.699187，ACC：0.619718，AUC：0.605149,MCC：0.213009,F1：0.679842
[[50 41]
 [40 82]]
敏感度TPR：0.555556，特异度TNR：0.666667，ACC：0.619718，AUC：0.609259,MCC：0.221902,F1：0.669388
[[40 33]
 [50 90]]
敏感度TPR：0.444444，特异度TNR：0.731707，ACC：0.610329，AUC：0.585095,MCC：0.183331,F1：0.684411
[[39 45]
 [51 78]]
敏感度TPR：0.433333，特异度TNR：0.634146，ACC：0.549296，AUC：0.533740,MCC：0

In [130]:
from abc import ABC, abstractmethod
import numpy as np
#Q
def get_measure3(predictors, X, y):
    """
    Measures the QStatistic Diversity.
    :param predictors: <list> Decision Trees
    :param X: <numpy ndaray> Feature vectors
    :param y: <numpy array> Target feature
    :return: <float>
    """
    n_instances = X.shape[0]
    n_predictors = len(predictors)
    q_total = 0
    for i in range(0, n_predictors-1):
        for j in range(i+1, n_predictors):
            n = np.zeros((2, 2))
            for k in range(n_instances):
                #print(k)
                i_pred = predictors[i].predict(X[k])
                #print(i_pred)
                j_pred = predictors[j].predict(X[k])
                true_y = y[k]
                if i_pred == true_y:
                    if j_pred == true_y:
                        n[1][1] += 1
                    else:
                        n[1][0] += 1
                else:
                    if j_pred == true_y:
                        n[0][1] += 1
                    else:
                        n[0][0] += 1

            # Adding a one value to the variables which are zeros
            for k in range(2):
                for l in range(2):
                    if n[k][l] == 0:
                        n[k][l] += 1
            same = n[1][1] * n[0][0]
            diff = n[1][0] * n[0][1]
            q_ij = (same - diff) / (same + diff)
            q_total += q_ij

    q_av = 2 * q_total / (n_predictors * (n_predictors - 1))
    return q_av

In [126]:
a=[i for i in range(167)]
len(a)

166

In [124]:
x_test1=[]
for pre_data in x_test:
    x_test1.append((pre_data).reshape(1,-1))
x_test1=np.array(x_test1)
x_test1.shape

(266, 1, 167)

In [ ]:
max_index_new4

In [132]:
print(get_measure3(predictors=[rf4.estimators_[0],rf3.estimators_[0]],X=x_test1, y=test_labels))#name_index pre_data1

0.39807162534435264


In [102]:
all_pre=[]
for x in max_index_new4:
    pre=rf4.estimators_[x].predict_proba(x_test)
    all_pre.append(pre)

In [103]:
all_pre

[array([[0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0

In [64]:
pre_singe1=rf0.estimators_[9].predict_proba(x_test)
pre_singe2=rf0.estimators_[4].predict_proba(x_test)
pre_singe3=rf0.estimators_[0].predict_proba(x_test)

pre_singe4=rf1.estimators_[8].predict_proba(x_test)
pre_singe5=rf1.estimators_[3].predict_proba(x_test)
pre_singe6=rf1.estimators_[7].predict_proba(x_test)

pre_singe7=rf2.estimators_[6].predict_proba(x_test)
pre_singe8=rf2.estimators_[3].predict_proba(x_test)
pre_singe9=rf2.estimators_[1].predict_proba(x_test)

pre_singe10=rf3.estimators_[9].predict_proba(x_test)
pre_singe11=rf3.estimators_[8].predict_proba(x_test)
pre_singe12=rf3.estimators_[7].predict_proba(x_test)

pre_singe13=rf4.estimators_[6].predict_proba(x_test)
pre_singe14=rf4.estimators_[8].predict_proba(x_test)
pre_singe15=rf4.estimators_[0].predict_proba(x_test)


In [80]:
pre_singe=pre_singe1+pre_singe2+pre_singe3+pre_singe4+pre_singe5+pre_singe6+pre_singe7+pre_singe8+pre_singe9
+pre_singe10+pre_singe11+pre_singe12+pre_singe13+pre_singe14+pre_singe15
pre=pre_singe/15.0

In [82]:
aa=analyze(pre,test_labels)

[[ 53  36]
 [ 55 122]]
敏感度TPR：0.490741，特异度TNR：0.772152，ACC：0.657895，AUC：0.710443,MCC：0.273613,F1：0.728358
